In [7]:
import os
import csv
import requests
import json
import pygal_maps_world.maps
from OTXv2 import OTXv2
from OTXv2 import IndicatorTypes
from datetime import datetime, timedelta
from dotenv import load_dotenv
load_dotenv()

otx = OTXv2(os.environ.get('AlienVault_API_Key'))

mtime = (datetime.now() - timedelta(days=90)).isoformat()

#Get all pulses modified by AlienVault/AlienVaultBots in the past 3 months
pulses = otx.getall(modified_since=mtime)
indicators = []
print(len(pulses))

#Get indicators with indicator_type=ipv4
#Sort by threat type
CNs = []
legend = {"scanning_host":[], "malware_hosting":[], "exploit_source":[], "command_and_control":[]}
for i in range(0, len(pulses)):
    pulseindicator = pulses[i]["indicators"]
    for indicator in pulseindicator:
        if indicator["type"] == "IPv4":
            if indicator["role"] != "scanning_host":
                legend["scanning_host"].append(indicator["indicator"])
            elif indicator["role"] != "malware_hosting":
                legend["malware_hosting"].append(indicator["indicator"])
            elif indicator["role"] != "exploit_source":
                legend["exploit_source"].append(indicator["indicator"])
            elif indicator["role"] != "command_and_control":
                legend["command_and_control"].append(indicator["indicator"])
print(len(legend["scanning_host"]))
print(len(legend["malware_hosting"]))
print(len(legend["exploit_source"]))
print(len(legend["command_and_control"]))


5779
34692
40
0
0


In [9]:
from collections import Counter

#Get indicator origin country
CNs = []
for i in range(0, len(legend["scanning_host"])):
    url = f'https://api.iplocation.net/?ip={legend["scanning_host"][i]}'
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        # Extract the country code from the response
        CNs.append(data.get('country_code2'))

top10 = Counter(CNs).most_common(10)
print(top10)

list_top10 = list(top10)
for i in range(0, len(top10)):
    top10[i] = (top10[i][0].lower(), top10[i][1])

# create a world map
worldmap = pygal_maps_world.maps.World()
  
# set the title of the map
worldmap.title = 'Top 10 Location of Reported Scanning IPs in the past month'
  
# adding the countries
worldmap.add(f'{top10[0][0]}', { top10[0][0] : top10[0][1]}, 
             f'{top10[1][0]}', { top10[1][0] : top10[1][1]}, 
             f'{top10[2][0]}', { top10[2][0] : top10[2][1]}, 
             f'{top10[3][0]}', { top10[3][0] : top10[3][1]}, 
             f'{top10[4][0]}', { top10[4][0] : top10[4][1]},
             f'{top10[5][0]}', { top10[5][0] : top10[5][1]},
             f'{top10[6][0]}', { top10[6][0] : top10[6][1]}, 
             f'{top10[7][0]}', { top10[7][0] : top10[7][1]}, 
             f'{top10[8][0]}', { top10[8][0] : top10[8][1]},  
             f'{top10[9][0]}', { top10[9][0] : top10[9][1]})
worldmap.render_to_file('top10scanners.svg')

CNs = []
for i in range(0, len(legend["malware_hosting"])):
    url = f'https://api.iplocation.net/?ip={legend["malware_hosting"][i]}'
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        # Extract the country code from the response
        CNs.append(data.get('country_code2'))

top10 = Counter(CNs).most_common(10)
print(top10)

list_top10 = list(top10)
for i in range(0, len(top10)):
    top10[i] = (top10[i][0].lower(), top10[i][1])

# create a world map
worldmap = pygal_maps_world.maps.World()
  
# set the title of the map
worldmap.title = 'Top 10 Location of Reported Malware Hosting IPs in the past month'
  
# adding the countries
worldmap.add(f'{top10[0][0]}', { top10[0][0] : top10[0][1]}, 
             f'{top10[1][0]}', { top10[1][0] : top10[1][1]}, 
             f'{top10[2][0]}', { top10[2][0] : top10[2][1]}, 
             f'{top10[3][0]}', { top10[3][0] : top10[3][1]}, 
             f'{top10[4][0]}', { top10[4][0] : top10[4][1]},
             f'{top10[5][0]}', { top10[5][0] : top10[5][1]},
             f'{top10[6][0]}', { top10[6][0] : top10[6][1]}, 
             f'{top10[7][0]}', { top10[7][0] : top10[7][1]}, 
             f'{top10[8][0]}', { top10[8][0] : top10[8][1]},  
             f'{top10[9][0]}', { top10[9][0] : top10[9][1]})
  
worldmap.render_to_file('top10malwarehosts.svg')